In [1]:
import pandas as pd
import numpy as np
import sqlalchemy as sa
import sqlite3
from sqlite3 import Error
from datetime import date, datetime, timezone, timedelta
from dateutil.parser import parse

import os
import glob
import re
import csv

In [2]:
def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by the db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
    except Error as e:
        print(e)

    return conn

In [3]:
connection = create_connection("NYC_YellowCab_data_filtered.sqlite3")

In [4]:
qry_tbl = '''
CREATE TABLE IF NOT EXISTS nyc_tripdata (
      VendorID INTEGER
    , tpep_pickup_datetime TIMESTAMP
    , tpep_dropoff_datetime TIMESTAMP
    , passenger_count INTEGER
    , trip_distance REAL
    , RatecodeID INTEGER
    , store_and_fwd_flag TEXT
    , PULocationID INTEGER
    , DOLocationID INTEGER
    , payment_type INTEGER
    , fare_amount REAL
    , extra REAL
    , mta_tax REAL
    , tip_amount REAL
    , tolls_amount REAL
    , improvement_surcharge REAL
    , total_amount REAL
    , congestion_surcharge REAL
);
'''

cursor = connection.cursor()
cursor.execute(qry_tbl)

In [5]:
for trip_file in glob.glob(r'[0-9][0-9][0-9][0-9]_Yellow_Taxi_Trip_Data.csv'):
    print(f"Processing file {trip_file}")
    fh = open(trip_file, encoding='utf-8')
    txt_records = csv.reader(fh)
    txt_records.__next__()

    sql_insert_records = "INSERT INTO nyc_tripdata (VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge) VALUES(?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)"
    cursor = connection.executemany(sql_insert_records, txt_records)
    print(f"Inserted {cursor.rowcount} rows")
    connection.commit()

Processing file 2019_Yellow_Taxi_Trip_Data.csv
Inserted 84399019 rows


In [6]:
qry_tbl2 = '''
CREATE TABLE IF NOT EXISTS taxi_zone_lookup (
      LocationID INTEGER
    , Borough TEXT
    , Zone TEXT
    , service_zone TEXT
);
'''

cursor = connection.cursor()
cursor.execute(qry_tbl2)

In [7]:
zone_file = "taxi_zone_lookup.csv"

fh = open(zone_file, encoding='utf-8')
print(f"Processing file {zone_file}")
txt_records = csv.reader(fh)
txt_records.__next__()

sql_insert_records = "INSERT INTO taxi_zone_lookup (LocationID, Borough, Zone, service_zone) VALUES(?, ?, ?, ?)"
cursor = connection.executemany(sql_insert_records, txt_records)
print(f"Inserted {cursor.rowcount} rows")
connection.commit()

Processing file taxi_zone_lookup.csv
Inserted 265 rows


In [8]:
qry_tbl3 = '''
CREATE TABLE nyc_tripdata_filtered (
	  VendorID INTEGER NOT NULL
	, tpep_pickup_datetime TIMESTAMP
	, tpep_dropoff_datetime TIMESTAMP
	, passenger_count INTEGER
	, trip_distance REAL
	, RatecodeID INTEGER
	, store_and_fwd_flag TEXT
	, PULocationID INTEGER
	, PUBorough TEXT
	, PUZone TEXT
	, DOLocationID INTEGER
	, DOBorough TEXT
	, DOZone TEXT
	, payment_type INTEGER
	, fare_amount REAL
	, extra REAL
	, mta_tax REAL
	, tip_amount REAL
	, tolls_amount REAL
	, improvement_surcharge REAL
	, total_amount REAL
	, congestion_surcharge REAL
);
'''

cursor = connection.cursor()
cursor.execute(qry_tbl3)

In [9]:
qry_tbl4 = '''
INSERT INTO nyc_tripdata_filtered
SELECT
	  trip.VendorID
	, trip.tpep_pickup_datetime
	, trip.tpep_dropoff_datetime
	, trip.passenger_count
	, trip.trip_distance
	, trip.RatecodeID
	, trip.store_and_fwd_flag
	, trip.PULocationID
	, lkpp.Borough AS PUBorough
	, lkpp.Zone AS PUZone
	, trip.DOLocationID
	, lkpd.Borough AS DOBorough
	, lkpd.Zone AS DOZone
	, trip.payment_type
	, trip.fare_amount
	, trip.extra
	, trip.mta_tax
	, trip.tip_amount
	, trip.tolls_amount
	, trip.improvement_surcharge
	, trip.total_amount
	, trip.congestion_surcharge
FROM nyc_tripdata AS trip
INNER JOIN taxi_zone_lookup AS lkpp
	ON lkpp.LocationID = trip.PULocationID
INNER JOIN taxi_zone_lookup AS lkpd
	ON lkpd.LocationID = trip.DOLocationID
WHERE lkpp.Borough IN ('Brooklyn', 'Queens')
AND lkpd.Borough IN ('Brooklyn', 'Queens', 'Manhattan', 'Bronx');
'''

cursor = connection.cursor()
cursor.execute(qry_tbl4)

In [10]:
connection = create_connection("NYC_YellowCab_selected_boroughs.sqlite3")

In [11]:
qry_tbl5 = "DROP TABLE IF EXISTS nyc_tripdata;"
cursor = connection.cursor()
cursor.execute(qry_tbl5)

In [12]:
qry_tbl6 = "DROP TABLE IF EXISTS taxi_zone_lookup;"
cursor = connection.cursor()
cursor.execute(qry_tbl6)

In [13]:
qry_tbl7 = "ALTER TABLE nyc_tripdata_filtered RENAME TO nyc_tripdata;"
cursor = connection.cursor()
cursor.execute(qry_tbl7)

In [14]:
connection.commit()

In [15]:
connection.close()

In [16]:
os.rename("NYC_YellowCab_data_filtered.sqlite3", "yellow_cab_selected_boroughs.sqlite3")